In [1]:
# !pip install beautifulsoup4

In [74]:
import requests
import csv
import time
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
url_base = "https://hbr.org"

# Principais textos topico

## Pega topicos

In [75]:
url = "https://hbr.org/topics"

# Headers to mimic a browser visit
headers = {'User-Agent': 'Mozilla/5.0'}

# Returns a requests.models.Response object
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')

topicos = soup.findAll('a')
topicos = pd.DataFrame([[topicos[i].contents[0], topicos[i]['href']] for i in range(len(topicos))], columns = ['topico','link'])
topicos = topicos[topicos.link.str.contains('/topic/')]
topicos['link_completo'] =url_base + topicos['link']
topicos= topicos.reset_index(drop = True)


topicos.to_csv('lista_topicos.csv', index = False, sep = '#' )


## Pega a lista de links

tabela = pd.DataFrame()
for i  in tqdm(range(topicos.shape[0])): 
    url = topicos[topicos.index==i].link_completo[i]
    # Headers to mimic a browser visit
    headers = {'User-Agent': 'Mozilla/5.0'}

    # Returns a requests.models.Response object
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')

    website = soup.findAll("stream-item")
    for element in website:
        link =soup.find_all("stream-item", {"class": "stream-item"})
        for tag in link:
            websitelink = tag.get("data-url")
            websitenome = tag.get("data-title")
            retorno = pd.DataFrame([[websitenome ,websitelink]])
            tabela = pd.concat([tabela,retorno])

tabela.columns =  ['titulo','link']
tabela = tabela.drop_duplicates()
tabela = tabela[~tabela.link.str.contains('/product/')]
tabela = tabela.reset_index(drop = True)
tabela['link_completo'] =url_base + tabela['link']

tabela.to_csv('lista_textos_por_topico.csv', index = False, sep = '#' )

## Vai na pagina do texto

tabela = tabela[~tabela.link.str.contains('video')]
tabela = tabela[~tabela.link.str.contains('sponsored')]
tabela = tabela[~tabela.link.str.contains('podcast')]
tabela = tabela[~tabela.link.str.contains('data-visuals')]

for i in tqdm(range(tabela.shape[0])):
    headers = {'User-Agent': 'Mozilla/5.0'}
    # Returns a requests.models.Response object]

    try:
        page = requests.get(tabela[tabela.index==i].link_completo[i], headers=headers)
#         print(i)
        soup = BeautifulSoup(page.text, 'html.parser')
        corpo_texto = soup.findAll("p")
        tabela.loc[tabela.index[i], 'retorno'] = str(corpo_texto)
    except:
        print('deu erro no' + str (i))

link_tratado = tabela['link'].str.split('/',regex= True,expand=True)
link_tratado.columns = ['vazio','ano','mes','titulo','vazio2' ]
link_tratado['safra'] = link_tratado['ano'] +link_tratado['mes']
tabela['safra'] = link_tratado['safra']
tabela = tabela.drop(['link_completo'],axis=1).sort_values(['safra']).reset_index(drop = True)

tabela.to_csv('artigos_por_topico_hbr_all.csv', sep = '#', index = False)

# Revistas mensais

In [133]:
def retorna_safras (ano_ini, ano_fim):
    # ano >2000 é BRYYSS BR1501
    # ano 2000 era BRYSS BR001-BR006
    # ano < 2000 era 3YYS
    if int(ano_ini) > 55:
        if int(ano_fim) >55:
            anos = ["{:02d}".format(i) for i in range(ano_ini,ano_fim+1)]
            meses= ['1','2','3','4','5','6']
            safras = []
            for ano in anos:
                for mes in meses:
                    retorno = '3' +  ano + mes
                    safras = safras + [retorno]
        elif int(ano_fim) ==0:
            anos = ["{:02d}".format(i) for i in range(ano_ini,100)]
            meses= ['1','2','3','4','5','6']
            safras = []
            for ano in anos:
                for mes in meses:
                    retorno = '3' +  ano + mes
                    safras = safras + [retorno]
            meses= ['01','02','03','04','05','06']
            for mes in meses:
                retorno = 'BR' + '0' + mes
                safras = safras + [retorno]                    
        else:
            anos = ["{:02d}".format(i) for i in range(ano_ini,99+1)]
            meses= ['1','2','3','4','5','6']
            safras = []
            for ano in anos:
                for mes in meses:
                    retorno = '3' +  ano + mes
                    safras = safras + [retorno]
            anos = ["{:02d}".format(i) for i in range(2,ano_fim+1)]        
            meses= ['01','02','03','04','05','06']
            for mes in meses:
                retorno = 'BR' + '0' + mes
                safras = safras + [retorno]            
            for ano in anos:
                for mes in meses:
                    retorno = 'BR' +  ano + mes
                    safras = safras + [retorno]
                    
    if int(ano_ini) <= 55:
        if int(ano_ini) ==0:
            anos = ["{:02d}".format(i) for i in range(2,ano_fim+1)]        
            meses= ['01','02','03','04','05','06']
            safras = []
            for mes in meses:
                retorno = 'BR' + '0' + mes
                safras = safras + [retorno]            
            for ano in anos:
                for mes in meses:
                    retorno = 'BR' +  ano + mes
                    safras = safras + [retorno]
        else:
                 
            anos = ["{:02d}".format(i) for i in range(ano_ini,ano_fim+1)]# anos > 2000
            meses= ['01','02','03','04','05','06']
            safras = []
            for ano in anos:
                for mes in meses:
                    retorno = 'BR' + ano + mes
                    safras = safras + [retorno]
    return safras

In [180]:
safras = retorna_safras (22, 23)
url_base = "https://hbr.org"
# Headers to mimic a browser visit
headers = {'User-Agent': 'Mozilla/5.0'}

tabela = pd.DataFrame()
for safra in tqdm(safras):
    url = "https://hbr.org/archive-toc/" + safra

    # Returns a requests.models.Response object
    try:
        page = requests.get(url, headers=headers)
    except:
        print('erro no ' + safra + ' no get')
    try:
        soup = BeautifulSoup(page.text, 'html.parser')
        website = soup.findAll("stream-item")
        for element in website:
            link =soup.find_all("stream-item", {"class": "stream-item"})
            for tag in link:
                websitelink = tag.get("data-url")
                websitenome = tag.get("data-title")
                retorno = [[websitenome ,websitelink,safra]]
                tabela = tabela.append(retorno)
    except:
        print('erro no ' + safra + ' no resto')
        
tabela.columns =  ['titulo','link','safra']
tabela = tabela.drop_duplicates()
tabela = tabela[~tabela.link.str.contains('/product/')]
tabela['link_completo'] =url_base + tabela['link']

tabela = tabela.reset_index(drop = True)
for i in tqdm(range(tabela.shape[0])):
    headers = {'User-Agent': 'Mozilla/5.0'}
    # Returns a requests.models.Response object

    try:
        page = requests.get(tabela[tabela.index==i].link_completo[i], headers=headers)
#         print(i)
        soup = BeautifulSoup(page.text, 'html.parser')
        corpo_texto = soup.findAll("p")
        tabela.loc[tabela.index[i], 'retorno'] = str(corpo_texto)
    except:
        print('deu erro no' + str (i) + ' para visitar a pagina')

link_tratado = tabela['link'].str.split('/',regex= True,expand=True)
link_tratado.columns = ['vazio','ano','mes','titulo' ]
link_tratado['safra'] = link_tratado['ano'] +link_tratado['mes']
tabela['safra'] = link_tratado['safra']

tabela = tabela.drop(['link_completo'],axis=1).sort_values(['safra']).reset_index(drop = True)


tabela.to_csv('revistas_historicas_hbr_'+safras[0]+'_'+safras[-1]+'.csv', sep = '#', index = False)

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:2

  8%|██████▉                                                                            | 1/12 [00:01<00:11,  1.05s/it]C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:2

C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:2

...

C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:2

 67%|███████████████████████████████████████████████████████▎                           | 8/12 [00:07<00:03,  1.05it/s]C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pa

C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:2

C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:2

C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tabela = tabela.append(retorno)
C:\Users\felip\AppData\Local\Temp\ipykernel_20752\518304638.py:2

# Consolida

In [110]:
textos_topicos = pd.read_csv('artigos_por_topico_hbr_all.csv', sep = '#')

textos_revistas = pd.read_csv('revistas_historicas_hbr_5601_2303.csv', sep = '#')
textos_revistas = textos_revistas[['titulo', 'link', 'retorno', 'safra']]

todos_textos = textos_revistas.append(textos_topicos).drop_duplicates().sort_values(['safra','titulo']).reset_index(drop = True)

In [132]:

todos_textos.to_csv('todos_textos_hbr_ate_202303.csv', sep = '#', index = False)

In [136]:
todos_textos

,titulo,link,retorno,safra
0,Thinking Ahead: Power Tactics,/1956/11/thinking-ahead-power-tactics,"[<p>Beneath the general principles, attitudes,...",195611
1,Listening to People,/1957/09/listening-to-people,[<p>Recently the top executives of a major man...,195709
2,Management in the 1980’s,/1958/11/management-in-the-1980s,[<p>Over the last decade a new technology has ...,195811
3,The Silent Language in Overseas Business,/1960/05/the-silent-language-in-overseas-business,"[<p>With few exceptions, Americans are relativ...",196005
4,Management of Differences,/1960/11/management-of-differences,[<p>The manager often experiences his most unc...,196011
...,...,...,...,...
6659,The Founders of Bitty & Beau’s Coffee on Build...,/2023/05/the-founders-of-bitty-and-beaus-coffe...,[<p>As the parents of two children with Down s...,202305
6660,The High Cost of Neglecting Low-Wage Workers,/2023/05/the-high-cost-of-neglecting-low-wage-...,[<p>Many companies blame outside factors for t...,202305
6661,The Leadership Odyssey,/2023/05/the-leadership-odyssey,[<p>A paradox of business is that while leader...,202305
6662,The Obstacles to Creating Good Jobs,/2023/05/the-obstacles-to-creating-good-jobs,[<p>The conventional wisdom in retail and othe...,202305


In [146]:
todos_textos[todos_textos.retorno.str.contains('span')]

,titulo,link,retorno,safra
1,Listening to People,/1957/09/listening-to-people,[<p>Recently the top executives of a major man...,195709
4,Management of Differences,/1960/11/management-of-differences,[<p>The manager often experiences his most unc...,196011
5,The Mystique of Super-Salesmanship,/1961/03/the-mystique-of-super-salesmanship,[<p>In contrast to the advances of modern indu...,196103
6,Businessmen in Power,/1961/09/businessmen-in-power,[<p>In all of the discussion about the explosi...,196109
7,Big Business and the National Purpose,/1962/03/big-business-and-the-national-purpose,[<p>American big business and its executives f...,196203
...,...,...,...,...
6659,The Founders of Bitty & Beau’s Coffee on Build...,/2023/05/the-founders-of-bitty-and-beaus-coffe...,[<p>As the parents of two children with Down s...,202305
6660,The High Cost of Neglecting Low-Wage Workers,/2023/05/the-high-cost-of-neglecting-low-wage-...,[<p>Many companies blame outside factors for t...,202305
6661,The Leadership Odyssey,/2023/05/the-leadership-odyssey,[<p>A paradox of business is that while leader...,202305
6662,The Obstacles to Creating Good Jobs,/2023/05/the-obstacles-to-creating-good-jobs,[<p>The conventional wisdom in retail and othe...,202305


In [147]:
soup = BeautifulSoup(todos_textos['retorno'][1], 'html.parser')

In [148]:
soup

[<p>Recently the top executives of a major manufacturing plant in the Chicago area were asked to survey the role that listening plays in their work. Later, an executive seminar on listening was held. Here are three typical comments made by participants:</p>, <p>These comments reflect part of an awakening that is taking place in a number of management circles. Business is tied together by its systems of communication. This communication, businessmen are discovering, depends more on the spoken word than it does on the written word; and the effectiveness of the spoken word hinges not so much on how people talk as on how they listen.</p>, <p>It can be stated, with practically no qualification, that people in general do not know how to listen. They have ears that hear very well, but seldom have they acquired the necessary aural skills which would allow those ears to be used effectively for what is called <em>listening</em>.</p>, <p>For several years we have been testing the ability of peopl